In [18]:
from pygsp import graphs, filters
import numpy as np

In [29]:
adjacency_matrix = np.load("processed_data/coexpression_adjacency.npy")
node_indices = np.load("processed_data/coexpression_node_indices.npy")
expression_data = np.load("processed_data/expression_data.pkl")

In [28]:
G = graphs.Graph(adjacency_matrix)

28367